# Preprocess audio (includes normalization) (ouput-folder: preprocessed_audio)
Output is used in MFCC

In [ ]:
import os
import numpy as np
from scipy.io import wavfile

def preprocess_audio_scipy(file_path, output_directory, target_sample_rate=22000, target_duration=2.0):
    """
    Preprocess audio using SciPy: convert to mono, downsample, trim/pad to 2 seconds, and normalize.
    Args:
        file_path (str): Path to the input audio file.
        output_directory (str): Directory to save the preprocessed file.
        target_sample_rate (int): Desired sampling rate for the audio.
        target_duration (float): Target duration in seconds.
    """
    # Load the audio file
    sample_rate, data = wavfile.read(file_path)

    # Convert to mono if stereo
    if len(data.shape) == 2:
        data = np.mean(data, axis=1).astype(data.dtype)

    # Downsample to the target sample rate
    if sample_rate != target_sample_rate:
        num_samples = int(len(data) * target_sample_rate / sample_rate)
        data = np.interp(
            np.linspace(0, len(data), num_samples, endpoint=False),
            np.arange(len(data)),
            data
        )
        sample_rate = target_sample_rate

    # Trim or pad the audio to the target duration
    target_length = int(sample_rate * target_duration)
    if len(data) > target_length:
        data = data[:target_length]
    elif len(data) < target_length:
        data = np.pad(data, (0, target_length - len(data)), mode='constant')

    # Normalize to zero mean and unit variance
    mean = np.mean(data)
    std = np.std(data)
    if std > 0:
        data = (data - mean) / std

    # Save the preprocessed file
    os.makedirs(output_directory, exist_ok=True)
    output_file = os.path.join(output_directory, os.path.basename(file_path))
    wavfile.write(output_file, sample_rate, data.astype(np.float32))
    print(f"Saved preprocessed file to: {output_file}")

def process_dataset_scipy(input_directory, output_directory):
    """
    Process all audio files in a directory using SciPy.
    Args:
        input_directory (str): Directory containing the original audio files.
        output_directory (str): Directory to save preprocessed files.
    """
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".wav"):
            file_path = os.path.join(input_directory, file_name)
            preprocess_audio_scipy(file_path, output_directory)

# Define input and output directories
input_directory = "/Users/jakob/Downloads/IDMT_Traffic/audio"
output_directory = "/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio"

# Process the dataset
process_dataset_scipy(input_directory, output_directory)